![EUROPE](http://nkpxv44v81c37wr753vdgve1.wpengine.netdna-cdn.com/wp-content/uploads/2015/10/Fotolia_44254321_Subscription_XXL-1000x250.jpg)

# WHERE TO RETIRE AT 50?

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

# Table of contents
1. [Research](#research)
2. [Executive Summary](#summary)
    1. [Sub paragraph](#subparagraph1)
3. [Data](#data)
4. [Web App](#app)
5. [Conclusions](#conclusions)
6. [Appendix](#appendix)


## Imports/Setup <a name="introduction"></a>

## RESEARCH <a name="research"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

## Executive Summary<a name="summary"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

## Data<a name="data"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

In [14]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize

In [2]:
url = 'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/urb_cpop1?precision=1&indic_ur=DE1001V'
data = requests.get(url)

parsed_data = data.json()
# drilling down to city name level
cities = parsed_data['dimension']
pop = parsed_data['value']
city2 = cities['cities']
city3 = city2['category']
city4 = city3['label']
# printing city names
for k in city4.values():
    parsed_city = k
for x in pop.values():
    parsed_pop = x

In [3]:
L_Map = pd.read_csv('retire.csv',sep=',')
L_Map.head()

,City,Location,Country,Average Rainfall,Average Temperature,Deaths per year under 65 avg,Deaths per year under 65 2050,Median Population Age,Median Population Age 2050,Homicides 2050,Economically Active Population,Population,Economically Active %,Cinema Attendance,Number of Libraries
0,Berlin,"(52.51,13.40)",Germany,562,28.0800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tallinn,"(59.43,24.75)",Estonia,763,38.1350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gdansk,"(54.34,18.64)",Poland,541,27.0625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Prague,"(50.07,14.43)",Czech Republic,408,20.4050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Munich,"(48.14,11.57)",Germany,941,47.0490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Maths to convert longtitude and latitude to Mercator for displaying on map
import math
from ast import literal_eval
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x,y)

In [5]:
# Testing Berlin Coordinates
merc('(52.51,13.40)')

(1491681.1766298658, 6892870.554499138)

In [6]:
L_Map['coords_x'] = L_Map['Location'].apply(lambda x: merc(x)[0])
L_Map['coords_y'] = L_Map['Location'].apply(lambda x: merc(x)[1])

In [7]:
L_Map[['Location','coords_x', 'coords_y','Average Rainfall','Country','City']].head()

,Location,coords_x,coords_y,Average Rainfall,Country,City
0,"(52.51,13.40)",1.491681e+06,6.892871e+06,562,Germany,Berlin
1,"(59.43,24.75)",2.755157e+06,8.273913e+06,763,Estonia,Tallinn
2,"(54.34,18.64)",2.074995e+06,7.234813e+06,541,Poland,Gdansk
3,"(50.07,14.43)",1.606340e+06,6.458407e+06,408,Czech Republic,Prague
4,"(48.14,11.57)",1.287967e+06,6.130178e+06,941,Germany,Munich


## Web App <a name="app"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

In [8]:
from bokeh.io import output_notebook, show
# reset_output()
from bokeh.plotting import figure
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Panel, Tabs, Slider
from bokeh.models.glyphs import HBar, VBar
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import ColumnDataSource, HoverTool, LabelSet, Label

In [12]:
source = ColumnDataSource(data=dict(
                        x=list(L_Map['coords_x']), 
                        y=list(L_Map['coords_y']),
                        sizes=list(L_Map['Average Temperature']),
                        country=list(L_Map['Country']),
                        city=list(L_Map['City']),
                        rain=list(L_Map['Average Rainfall'])))

hover = HoverTool(tooltips=[
    ("Average Rain (mm)", "@rain"),
    ("City", "@city"),
    ("Country", "@country"),
    
])

labels=LabelSet(x='x', y='y', text='rain', level='glyph',
         text_align='center', source=source)

# Rain Plot
p1 = figure(x_axis_type="mercator", 
           y_axis_type="mercator", 
           plot_width = 900, 
           plot_height = 600,
           tools = [hover, 'wheel_zoom','reset','save'])

p1.add_tile(CARTODBPOSITRON)

p1.circle(x = 'x',y = 'y',
         source = source,
         color = 'blue', 
         alpha = 0.5, 
         size = 'sizes')
tab1 = Panel(child=p1, title="Rain")

# Temp Plot
cities = list(L_Map['City'])
count = list(L_Map['Average Rainfall'])

# sorting the bars means sorting the range factors
sorted_city = sorted(cities, key=lambda x: count[cities.index(x)])

p2 = figure(x_range=sorted_city,
            plot_width = 900, 
            plot_height = 600, 
            tools=[hover, 'wheel_zoom','reset','save'])

p2.vbar(x="city", 
        top='rain',
        source =source,
        width=0.5)
p2.xaxis.major_label_orientation = 1


tab2 = Panel(child=p2, title="Temp")

# Crime Plot

p3 = figure(y_range=sorted_city, 
            plot_width = 900, 
            plot_height = 600, 
            tools=[hover, 'wheel_zoom','reset','save'])

p3.hbar(y='city', 
        right = 'rain',
        color ='orange',
        source =source,
        height=0.5)



tab3 = Panel(child=p3, title="Crime")

# Rain Plot
p4 = figure(x_axis_type="mercator", 
           y_axis_type="mercator", 
           plot_width = 900, 
           plot_height = 600,
           tools = [hover, 'wheel_zoom','reset','save'])

p4.add_tile(CARTODBPOSITRON)

p4.circle(x = 'x',y = 'y',
         source = source,
         color = 'green', 
         alpha = 0.5, 
         size = 'sizes')

tab4 = Panel(child=p4, title="Population")

tabs = Tabs(tabs=[ tab4, tab2, tab3, tab1 ])
show(tabs)
output_notebook ()

Loading BokehJS ...

## The Best City? <a name="best"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

In [32]:
Best = pd.read_csv('rank.csv',sep=',')

Best.head(10).style.highlight_min(axis=0, color='lightgreen')

,City,Weighted Rank
0,Berlin,1
1,Tallinn,2
2,Gdansk,3
3,Prague,4
4,Munich,5
5,Madrid,6
6,Tenerife,7
7,Warsaw,8
8,Lisbon,9
9,Ponta Delgada,10


## Conclusions<a name="conlusions"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

## Appendix <a name="appendix"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___